In [ ]:
import openai
from openai import OpenAI
client = OpenAI(api_key="Your_API_Key")
import re
import json
import pandas as pd

In [ ]:
def gpt(prompt: str, model='gpt-3.5-turbo-0613', temperature = 0, n=1):
    output = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=1000,
        n=n
    )
    choices = []
    choices.extend([choice.message.content for choice in output.choices])
    return choices

def proofread(text, model='gpt-3.5-turbo-0613', temperature = 0, n=1):
    # proofread the user's and designer's responses of goals before extracting embeddings
    prompt = f'''Proofread the given text, correcting spelling and grammar errors. Eliminate special characters used for ordering. Expand all abbreviations.
    <text>
    {text}

    <proofread text>
    '''
    return gpt(prompt=prompt, model=model, temperature=temperature, n=n)

def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# User embeddings

In [ ]:
dfu = pd.read_csv("/content/drive/MyDrive/mind_inference_experiment/Randomlized_24_User_Data.csv")
dfu

,Index,Category,Product,Comment,Post_or_Pre,Do-goals,Motor-goals,Be-goals,FPN1_Autonomy,FPN2_Beauty,...,FPN4_Community,FPN5_Competence,FPN6_Fitness,FPN7_Impact,FPN8_Morality,FPN9_Purpose,FPN10_Recognition,FPN11_Relatedness,FPN12_Security,FPN13_Stimulation
0,0,2,Luggage,The luggage is small in size to fit as a cabbi...,post,I aim to carry my materials compactly and comf...,"The bag should be an adequate size, big enough...",I hope to be relaxed and in a peaceful state w...,4,4,...,3,3,1,3,1,5,2,1,5,1
1,1,2,travel adapter (American to European wall outl...,It is small and lightweight which made it good...,post,I aim to use this adaptor to charge my laptop ...,"By plugging the adaptor into the wall, my lapt...",I aspire to conduct engineering research in Eu...,5,1,...,1,5,1,4,1,4,1,1,4,5
2,2,2,Portable charger,"This portable charger is the perfect size, and...",post,I am to recharge my phone to 100% with this po...,By plugging in a usb/usb-c charger into this p...,I aspire to be in control and feel secure with...,4,1,...,1,2,2,1,1,4,1,1,5,2
3,3,3,desktop computer (Mac Mini),"The set up was easy and straightforward, and t...",post,I aim to complete multiple tasks quickly on th...,I can complete multiple tasks on the desktop b...,I aspire to be more efficient at my work,3,2,...,1,5,1,3,1,1,1,1,1,1
4,4,3,laptop,The laptop is very sleek and had a long batter...,post,I aim to use the laptop's apps and browser for...,"I open the laptop, open VSCode, and start typi...",I aspire to increase my knowledge and improve ...,2,1,...,1,5,1,5,2,4,3,1,2,1
5,5,1,Sponge,"The sponge is functional enough when washing, ...",post,The goal is to ensure dishes are clean of food...,I should be able to apply water (warm) and dis...,"I hope to have a clean/hygenic environment, es...",2,2,...,1,3,4,1,1,1,1,1,4,1
6,6,2,Portable charger,"I like this charger, it has a long battery lif...",post,I aim to charge my phone on the go. I aim to b...,I need to make sure my portable charger is ful...,I aspire to stay connected; I need to feel saf...,4,1,...,5,1,1,1,1,3,1,1,5,1
7,7,3,Razer Naga Trinity Mouse,The Trinity is a great replacement and upgrade...,post,I aim to improve my gameplay efficiency in WoW...,By programming the 12-button side panel and le...,I aspire to attain higher achievement in my pu...,4,5,...,2,5,1,1,1,3,4,2,1,4
8,8,1,Swiffer wet mop,The wet mop is useful in cleaning up some basi...,post,I aim to clean my floor; I aim to remove dried...,"By loading the wet wipes onto the Swiffer mop,...",I aspire to have impact on the state of my flo...,3,1,...,1,3,4,5,1,3,1,1,1,2
9,9,2,Travel Adapter,Can this adapter connect to my iphone and non ...,pre,1. I would like to be able to use non apple he...,"1. By inserting the new cable, I should be abl...",1. I aspire to be able to multi-task/ manage w...,5,1,...,2,5,4,5,3,5,5,1,4,3


In [ ]:
fpn_columns = [col for col in dfu.columns if col.startswith('FPN')]
dfu['fpns'] = dfu.apply(lambda row: [row[fpn] for fpn in fpn_columns], axis=1)
dfu['pr_dg'] = dfu['Do-goals'].apply(lambda x: proofread(x)[0])
dfu['pr_mg'] = dfu['Motor-goals'].apply(lambda x: proofread(x)[0])
dfu['pr_bg'] = dfu['Be-goals'].apply(lambda x: proofread(x)[0])
dfu['dg_embedding'] = dfu['pr_dg'].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
dfu['mg_embedding'] = dfu['pr_mg'].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
dfu['bg_embedding'] = dfu['pr_bg'].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
dfu.head()

,Index,Category,Product,Comment,Post_or_Pre,Do-goals,Motor-goals,Be-goals,FPN1_Autonomy,FPN2_Beauty,...,FPN11_Relatedness,FPN12_Security,FPN13_Stimulation,fpns,pr_dg,pr_mg,pr_bg,dg_embedding,mg_embedding,bg_embedding
0,0,2,Luggage,The luggage is small in size to fit as a cabbi...,post,I aim to carry my materials compactly and comf...,"The bag should be an adequate size, big enough...",I hope to be relaxed and in a peaceful state w...,4,4,...,1,5,1,"[4, 4, 5, 3, 3, 1, 3, 1, 5, 2, 1, 5, 1]",I aim to carry my materials compactly and comf...,"The bag should be an adequate size, big enough...",I hope to be relaxed and in a peaceful state w...,"[-0.024392396211624146, 0.027402115985751152, ...","[-0.048109713941812515, 0.01738385297358036, 0...","[-0.03234438970685005, 0.020991625264286995, -..."
1,1,2,travel adapter (American to European wall outl...,It is small and lightweight which made it good...,post,I aim to use this adaptor to charge my laptop ...,"By plugging the adaptor into the wall, my lapt...",I aspire to conduct engineering research in Eu...,5,1,...,1,4,5,"[5, 1, 1, 1, 5, 1, 4, 1, 4, 1, 1, 4, 5]",I aim to use this adapter to charge my laptop ...,"By plugging the adapter into the wall, my lapt...",I aspire to conduct engineering research in Eu...,"[-0.02032318338751793, 0.04584910348057747, -0...","[0.0001383212220389396, 0.04337985813617706, -...","[-0.03136971965432167, 0.03054780513048172, -0..."
2,2,2,Portable charger,"This portable charger is the perfect size, and...",post,I am to recharge my phone to 100% with this po...,By plugging in a usb/usb-c charger into this p...,I aspire to be in control and feel secure with...,4,1,...,1,5,2,"[4, 1, 5, 1, 2, 2, 1, 1, 4, 1, 1, 5, 2]",I am going to recharge my phone to 100 percent...,By plugging in a USB/USB-C charger into this p...,I aspire to be in control and feel secure with...,"[-0.045712582767009735, 0.056636933237314224, ...","[-0.03238840773701668, 0.0747632384300232, -0....","[-0.0349658764898777, 0.04574235528707504, -0...."
3,3,3,desktop computer (Mac Mini),"The set up was easy and straightforward, and t...",post,I aim to complete multiple tasks quickly on th...,I can complete multiple tasks on the desktop b...,I aspire to be more efficient at my work,3,2,...,1,1,1,"[3, 2, 5, 1, 5, 1, 3, 1, 1, 1, 1, 1, 1]",I aim to complete multiple tasks quickly on th...,I can complete multiple tasks on the desktop b...,I aspire to be more efficient at my work.,"[-0.022806786000728607, 0.002680284669622779, ...","[0.004365907981991768, 0.03254324570298195, -0...","[-0.02478894405066967, 0.014668968506157398, -..."
4,4,3,laptop,The laptop is very sleek and had a long batter...,post,I aim to use the laptop's apps and browser for...,"I open the laptop, open VSCode, and start typi...",I aspire to increase my knowledge and improve ...,2,1,...,1,2,1,"[2, 1, 1, 1, 5, 1, 5, 2, 4, 3, 1, 2, 1]",I aim to use the laptop's applications and web...,"I open the laptop, open Visual Studio Code, an...",I aspire to increase my knowledge and improve ...,"[-0.025130746886134148, -0.0015777747612446547...","[0.030043108388781548, 0.024901222437620163, -...","[-0.036632269620895386, 0.015062123537063599, ..."


In [ ]:
dfu.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_users_ebdL.h5', key='user')

<ipython-input-10-0474ec865093>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['Product', 'Comment', 'Post_or_Pre', 'Do-goals', 'Motor-goals',
       'Be-goals', 'fpns', 'pr_dg', 'pr_mg', 'pr_bg', 'dg_embedding',
       'mg_embedding', 'bg_embedding'],
      dtype='object')]

  dfu.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_users_ebdL.h5', key='user')


# Designer embeddings

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/mind_inference_experiment/Designer_inference_result.csv")
df

,participant_id,comment_id,do-goal,motor-goal,be-goal,FPN1_Autonomy,FPN2_Beauty,FPN3_Comfort,FPN4_Community,FPN5_Competence,FPN6_Fitness,FPN7_Impact,FPN8_Morality,FPN9_Purpose,FPN10_Recognition,FPN11_Relatedness,FPN12_Security,FPN13_Stimulation
0,0,16,I aim to deeply clean my floors; I aim to get ...,Scrub the floors with the swiffer wipe without...,I wish to have a clean environment,1,4,5,1,1,1,1,1,1,1,1,1,1
1,1,7,I aim to charge my phone.\n I aim to use my po...,"By connecting my phone to my portable charger,...",I aspire to be secure and have control over my...,5,1,5,1,1,1,4,1,5,1,1,5,1
2,2,1,I am to travel with the luggage. I am to store...,I am to pick up the luggage to be able to stor...,I aspire to travel lightly and be able to brin...,5,2,5,1,4,1,1,1,2,1,1,2,1
3,3,18,I aim to hold a lot of stuff\n I aim to use my...,"By closing the zipper, I should be able to hol...",I aspire to bring a lot of stuff to my workpla...,1,1,4,1,1,1,1,1,1,1,1,5,1
4,4,12,I aim to clean my home\n I aim to clean multip...,"By swiping a wipe over dirty surfaces, I will ...",I aspire to have a clean home\n I aspire to cl...,2,3,4,1,2,1,1,1,1,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,15,5,I want to use the laptop for a long time (long...,I have to use my laptop on my daily task witho...,I want to have power/control on my tasks. \n I...,5,1,5,1,5,2,5,1,5,3,4,5,4
116,16,22,I aim to vacuum a college dorm\n I aim to use ...,"By plugging in the vacuum, I should be able to...",I aspire to maintain a clean space\n I aspire ...,4,1,4,1,2,1,1,1,3,1,1,1,3
117,17,13,I aim to clean dirty surfaces. I also aim to h...,"By operating it in the correct way, I should b...",I aspire to have a clean environment. I also a...,2,5,5,1,1,1,1,1,1,1,1,1,2
118,18,10,I aim to use a travel adapter when I'm on the ...,"By using different wires, I should be able to ...",I aspire for efficiency; I want a sense of sec...,4,1,4,1,4,1,3,1,5,1,1,5,1


In [ ]:
dfed = df.copy()
fpn_columns = [col for col in dfed.columns if col.startswith('FPN')]
dfed['fpns'] = dfed.apply(lambda row: [row[fpn] for fpn in fpn_columns], axis=1)
dfed['pr_dg'] = dfed['do-goal'].apply(lambda x: proofread(x)[0])
dfed['pr_mg'] = dfed['motor-goal'].apply(lambda x: proofread(x)[0])
dfed['pr_bg'] = dfed['be-goal'].apply(lambda x: proofread(x)[0])


In [ ]:
dfed['dg_embedding'] = dfed['pr_dg'].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
dfed['mg_embedding'] = dfed['pr_mg'].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
dfed['bg_embedding'] = dfed['pr_bg'].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
dfed.head()

,participant_id,comment_id,do-goal,motor-goal,be-goal,FPN1_Autonomy,FPN2_Beauty,FPN3_Comfort,FPN4_Community,FPN5_Competence,...,FPN11_Relatedness,FPN12_Security,FPN13_Stimulation,fpns,pr_dg,pr_mg,pr_bg,dg_embedding,mg_embedding,bg_embedding
0,0,16,I aim to deeply clean my floors; I aim to get ...,Scrub the floors with the swiffer wipe without...,I wish to have a clean environment,1,4,5,1,1,...,1,1,1,"[1, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",I aim to thoroughly clean my floors. I aim to ...,Scrub the floors with the Swiffer wipe without...,I wish to have a clean environment.,"[-0.030162597075104713, 5.7604604080552235e-05...","[-0.019896987825632095, -0.020356565713882446,...","[-5.169917858438566e-05, -0.03868808597326279,..."
1,1,7,I aim to charge my phone.\n I aim to use my po...,"By connecting my phone to my portable charger,...",I aspire to be secure and have control over my...,5,1,5,1,1,...,1,5,1,"[5, 1, 5, 1, 1, 1, 4, 1, 5, 1, 1, 5, 1]",I am going to charge my phone.\nI am going to ...,"By connecting my phone to my portable charger,...",I aspire to be secure and have control over my...,"[-0.049420397728681564, 0.051874492317438126, ...","[-0.028534354642033577, 0.06238867715001106, -...","[-0.03441281244158745, 0.014471872709691525, -..."
2,2,1,I am to travel with the luggage. I am to store...,I am to pick up the luggage to be able to stor...,I aspire to travel lightly and be able to brin...,5,2,5,1,4,...,1,2,1,"[5, 2, 5, 1, 4, 1, 1, 1, 2, 1, 1, 2, 1]",I am going to travel with the luggage. I am go...,I am going to pick up the luggage in order to ...,I aspire to travel lightly and be able to brin...,"[-0.0604461207985878, 0.0362388901412487, -0.0...","[-0.0752788707613945, 0.03023623302578926, -0....","[-0.038662139326334, 0.028425490483641624, -0...."
3,3,18,I aim to hold a lot of stuff\n I aim to use my...,"By closing the zipper, I should be able to hol...",I aspire to bring a lot of stuff to my workpla...,1,1,4,1,1,...,1,5,1,"[1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1]",I aim to hold a lot of stuff.\nI aim to use my...,"By closing the zipper, I should be able to hol...",I aspire to bring a lot of things to my workpl...,"[-0.04590984061360359, -0.003658713772892952, ...","[-0.027896016836166382, -0.004098790232092142,...","[-0.03845985233783722, 0.022248852998018265, -..."
4,4,12,I aim to clean my home\n I aim to clean multip...,"By swiping a wipe over dirty surfaces, I will ...",I aspire to have a clean home\n I aspire to cl...,2,3,4,1,2,...,1,3,1,"[2, 3, 4, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1]",I aim to clean my home.\nI aim to clean multip...,"By wiping a cloth over dirty surfaces, I will ...",I aspire to have a clean home.\nI aspire to cl...,"[-0.0346590094268322, -0.006865093484520912, -...","[-0.04825444892048836, -0.025451738387346268, ...","[-0.04781842604279518, 0.003751913085579872, -..."


In [ ]:
dfed.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_designers_ebdL.h5', key='designer')

<ipython-input-7-f115c66e4f83>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['do-goal', 'motor-goal', 'be-goal', 'fpns', 'pr_dg', 'pr_mg', 'pr_bg',
       'dg_embedding', 'mg_embedding', 'bg_embedding'],
      dtype='object')]

  dfed.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_designers_ebdL.h5', key='designer')


# Calculate designer similarity scores

In [4]:
import pandas as pd
import numpy as np

def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((np.array(vec1) - np.array(vec2)) ** 2))

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def normalize_needs_vector(elements):
    normalized_list = []
    for element in elements:
        try:
            int_element = int(element)
            normalized_element = (int_element - 1) / 4
            normalized_list.append(normalized_element)
        except ValueError:
            normalized_list.append(element)
    return normalized_list

In [5]:
dfu = pd.read_hdf('/content/drive/MyDrive/mind_inference_experiment/result_users_ebdL.h5', key='user')
dfd = pd.read_hdf('/content/drive/MyDrive/mind_inference_experiment/result_designers_ebdL.h5', key='designer')

In [11]:
for i in range(len(dfu)):
  id = int(dfu.loc[i,'Index'])
  base_dg, base_mg, base_bg, base_fpns = dfu.loc[i, ['dg_embedding','mg_embedding','bg_embedding','fpns']]
  for n in range(len(dfd)):
    if dfd.loc[n,'comment_id'] == id+1:
       hu_dg, hu_mg, hu_bg, hu_fpns = dfd.loc[n, ['dg_embedding','mg_embedding','bg_embedding','fpns']]
       cos_d, cos_m, cos_b = cosine_similarity(base_dg,hu_dg), cosine_similarity(base_mg,hu_mg), cosine_similarity(base_bg,hu_bg)
       cos_fpn = cosine_similarity(normalize_needs_vector(base_fpns),normalize_needs_vector(hu_fpns))
       euc_fpn = euclidean_distance(normalize_needs_vector(base_fpns),normalize_needs_vector(hu_fpns))
       dfd.loc[n, ["dg_cos","mg_cos","bg_cos","fpn_cos","fpn_euc"]] = cos_d, cos_m, cos_b, cos_fpn, euc_fpn

In [13]:
dfd.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_designers_ebdL_scores.h5', key='designer')

<ipython-input-13-2b839a3e56c2>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['do-goal', 'motor-goal', 'be-goal', 'fpns', 'pr_dg', 'pr_mg', 'pr_bg',
       'dg_embedding', 'mg_embedding', 'bg_embedding'],
      dtype='object')]

  dfd.to_hdf('/content/drive/MyDrive/mind_inference_experiment/result_designers_ebdL_scores.h5', key='designer')
